In [238]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
from pathlib import Path
import requests
import urllib.request
import json
import pprint
from dotenv import load_dotenv

%matplotlib inline

In [239]:
load_dotenv()
access_token = os.getenv("CQ_API_KEY")
headers = { 'Authorization': 'Bearer ' + access_token }
URL = "https://api.cryptoquant.com/v1/btc/exchange-flows/netflow?exchange=all_exchange&window=day&from=20190101"
response = requests.get(URL, headers=headers)

inflow_data = response.json()
#print(type(inflow_data))

#pprint.pprint(inflow_data)
inflow_df = pd.DataFrame(data=inflow_data['result']['data'])
inflow_df['date'] = pd.to_datetime(inflow_df['date'])
#inflow_df = inflow_df.set_index('date')
inflow_df.head(5)
#inflow_df.dtypes

,date,netflow_total
0,2021-01-19,-14814.09496357
1,2021-01-18,6798.66556348
2,2021-01-17,-5060.86295078
3,2021-01-16,-355.41891956
4,2021-01-15,17624.14937124


In [240]:
URL2 = "https://api.cryptoquant.com/v1/btc/market-data/price-usd?window=day&from=20190101"
response = requests.get(URL2, headers=headers)

price_data = response.json()
print(type(price_data))

#pprint.pprint(price_data)
price_df = pd.DataFrame(data=price_data['result']['data'])
price_df = price_df.drop(['price_usd_high','price_usd_low','price_usd_open'], axis = 1)
price_df['date'] = pd.to_datetime(price_df['date'])
#price_df = price_df.set_index('date')

price_df.head(5)



<class 'dict'>


,date,price_usd_close
0,2021-01-19,35888.901667
1,2021-01-18,36641.193333
2,2021-01-17,35830.791667
3,2021-01-16,35991.390000
4,2021-01-15,36761.818333


In [250]:
#column_1 = price_df["price_usd_close"]
#column_2 = inflow_df["netflow_total"]
#correlation = column_1.corr(column_2)
#print(correlation)


price_to_netflow_df = pd.concat([price_df,inflow_df['netflow_total']],ignore_index=False, axis=1, sort=False)
price_to_netflow_df = price_to_netflow_df.drop_duplicates()


price_to_netflow_df.head(10)

,date,price_usd_close,netflow_total
0,2021-01-19,35888.901667,-14814.09496357
1,2021-01-18,36641.193333,6798.66556348
2,2021-01-17,35830.791667,-5060.86295078
3,2021-01-16,35991.390000,-355.41891956
4,2021-01-15,36761.818333,17624.14937124
5,2021-01-14,39151.978333,-3318.76613269
6,2021-01-13,37383.333333,4514.32307716
7,2021-01-12,34036.688333,83.45243092
8,2021-01-11,35437.078333,5264.57279200
9,2021-01-10,38175.311667,9508.08974645


In [259]:
#price_to_netflow_df.plot(x="date", y=["price_usd_close", "netflow_total"])
#price_to_netflow_df.set_index('date').plot(figsize=(10,5), grid=True)

In [260]:
#fig, ax1 = plt.subplots()

#color = 'tab:red'
#ax1.set_xlabel('Date')
#ax1.set_ylabel('BTC Netflow', color=color)
#ax1.plot(price_to_netflow_df['date'], price_to_netflow_df['netflow_total'], color=color)
#ax1.tick_params(axis='y', labelcolor=color)

#ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

#color = 'tab:blue'
#ax2.set_ylabel('Price USD', color=color)  # we already handled the x-label with ax1
#ax2.plot(price_to_netflow_df['date'], price_to_netflow_df['price_usd_close'], color=color)
#ax2.tick_params(axis='y', labelcolor=color)

#fig.tight_layout()  # otherwise the right y-label is slightly clipped
#plt.show()